In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [8]:
!mkdir -p data results logs


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata


In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Todos:

 

1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






Run following in local Jupyter

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


Run following 2 cells for colab to import libraries

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


Run following if the library code is modified locally

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [5]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
#            'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
#            'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
#            'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
#            'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
#            'MAXR', 'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
#            'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
#            'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM', 'RBLX',
#            'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM', 'STNE','SWAV',
#            'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
#            'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
model = "gstock-4a-test"
tickers = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA', 
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-4b-test"
tickers = ['AAL', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN', 
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB', 
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK', 
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-4b-test"
tickers = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']

In [6]:
findata.EPOCHS=200
model = "etf-5a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX', 
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM', 
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)


In [10]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
22,SMH,97.67,0.88,1543.65,565.75,124.38,235.139999,0.89,0.76,1169.93,697.77,208.520004,0.68,0.64,903.13,455.04,195.630005,0.57,0.68,8.623529
10,IPO,22.26,0.93,302.32,257.97,28.00,25.620001,-0.08,0.71,229.15,178.52,27.969999,-0.00,0.59,218.49,169.21,24.200001,-0.14,-0.08,8.425000
0,ARKF,14.12,0.91,208.19,261.86,18.52,17.100000,-0.08,0.70,162.66,188.01,18.180000,-0.02,0.63,179.01,254.21,16.549999,-0.11,-0.08,7.975000
2,ARKW,38.25,0.93,708.46,669.22,48.84,45.439999,-0.07,0.71,371.93,587.59,49.320000,0.01,0.69,627.03,496.16,42.549999,-0.13,-0.07,8.300000
30,XITK,86.47,0.93,991.66,1034.88,107.65,100.339996,-0.07,0.69,683.73,581.11,109.120003,0.01,0.59,569.84,534.56,88.690002,-0.18,-0.07,8.300000
7,FPX,55.51,0.91,445.78,357.50,80.80,75.419998,-0.07,0.69,292.22,246.89,81.370003,0.01,0.68,304.31,443.90,70.489998,-0.13,-0.07,7.814286
40,XLV,72.26,0.90,366.35,259.97,133.59,125.830002,-0.06,0.77,387.73,168.03,133.429993,-0.00,0.64,312.35,156.82,118.400002,-0.11,-0.06,7.333333
37,XLP,45.46,0.89,186.74,126.28,77.18,72.339996,-0.06,0.73,156.66,80.30,76.290001,-0.01,0.63,213.46,122.27,69.970001,-0.09,-0.06,7.066667
42,XME,14.50,0.90,259.03,166.96,48.47,50.810001,0.05,0.74,209.01,182.19,50.849998,0.05,0.57,164.60,103.99,41.990002,-0.13,0.05,7.000000
20,ONLN,27.06,0.91,307.84,351.76,29.40,30.600000,0.04,0.76,277.76,270.05,31.309999,0.06,0.66,196.22,242.64,28.110001,-0.04,0.04,6.850000


In [ ]:
df1  = pipeline.runModelCombinedVolaR(tickers, model, mod, False, loss=lossfn)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False).head(50)


,,Name,Error,Accu,Buy,Sell,Total,Last,Pred,Gain,Alloc
Ticker,Target,,,,,,,,,,
PROSF,adjclose,RROnly,43.62,0.97,537.58,466.35,1003.94,71.27,81.980003,0.15,9.500000
ADYEY,adjclose,RROnly,6.69,0.96,158.37,129.34,287.71,15.88,14.230000,-0.10,9.200000
COTY,adjclose,RROnly,2.84,0.94,83.03,89.97,173.01,12.26,10.630000,-0.13,8.938462
NOK,adjclose,RROnly,2.47,0.94,27.42,36.66,64.09,4.90,5.450000,0.11,8.854545
STLA,adjclose,RROnly,5.24,0.94,111.36,84.21,195.57,18.86,16.700001,-0.11,8.854545
TKAMY,adjclose,RROnly,3.99,0.94,82.39,90.40,172.78,7.83,8.600000,0.10,8.800000
NU,adjclose,RROnly,3.47,0.93,14.38,26.09,40.47,4.66,5.220000,0.12,8.716667
CPG,adjclose,RROnly,0.72,0.90,62.35,36.14,98.49,7.78,5.590000,-0.28,8.642857
COVTY,adjclose,RROnly,11.97,0.92,125.21,160.04,285.25,20.17,22.510000,0.12,8.533333


In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV', 
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE', 
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE', 
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [11]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 

sending incremental file list
etf-5a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-5a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-5a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,640,235 bytes  received 73 bytes  19,280,616.00 bytes/sec
total size is 9,637,296  speedup is 1.00
